In [1]:
import sys
import os
sys.path.append('../')
import pathlib as pl
from SymEigen import *
from sympy import symbols
from project_dir import project_dir

Gen = EigenFunctionGenerator()
Gen.MacroBeforeFunction("__host__ __device__")


In [2]:
D, dHat, kappa, xi = symbols('D dHat kappa xi')
Cl = Gen.Closure(kappa, D, dHat, xi)
kB = - kappa * (D - xi * xi - 2 * xi * dHat - dHat * dHat) **2 * log((D-xi * xi)/(2 * xi * dHat + dHat * dHat))
kB

-(D - dHat**2 - 2*dHat*xi - xi**2)**2*log((D - xi**2)/(dHat**2 + 2*dHat*xi))

In [3]:
dkBdd = kB.diff(D)
dkBdd

-(2*D - 2*dHat**2 - 4*dHat*xi - 2*xi**2)*log((D - xi**2)/(dHat**2 + 2*dHat*xi)) - (D - dHat**2 - 2*dHat*xi - xi**2)**2/(D - xi**2)

In [4]:
ddkBddd = dkBdd.diff(D)
ddkBddd



-2*log((D - xi**2)/(dHat**2 + 2*dHat*xi)) - 2*(2*D - 2*dHat**2 - 4*dHat*xi - 2*xi**2)/(D - xi**2) + (D - dHat**2 - 2*dHat*xi - xi**2)**2/(D - xi**2)**2

In [5]:
s = f'''
// > Squared Version
// > D := d*d

{Cl("KappaBarrier",kB)}
{Cl("dKappaBarrierdD",dkBdd)}
{Cl("ddKappaBarrierddD",ddkBddd)}
'''
print(s)

f = open( project_dir() / 'src/uipc/backends/cuda/contact_system/contact_models/sym/codim_ipc_contact.inl', 'w')
f.write(s)
f.close()


// > Squared Version
// > D := d*d

template <typename T>
__host__ __device__ void KappaBarrier(T& R, const T& kappa, const T& D, const T& dHat, const T& xi)
{
/*****************************************************************************************************************************
Function generated by SymEigen.py 
Author: MuGdxy
GitHub: https://github.com/MuGdxy/SymEigen
E-Mail: lxy819469559@gmail.com
******************************************************************************************************************************
LaTeX expression:
//tex:$$R = - \left(D - \hat{d}^{2} - 2 \hat{d} \xi - \xi^{2}\right)^{2} \log{\left(\frac{D - \xi^{2}}{\hat{d}^{2} + 2 \hat{d} \xi} \right)}$$

Symbol Name Mapping:
kappa:
    -> {}
    -> Matrix([[kappa]])
D:
    -> {}
    -> Matrix([[D]])
dHat:
    -> {}
    -> Matrix([[dHat]])
xi:
    -> {}
    -> Matrix([[xi]])
*****************************************************************************************************************************/